# Amazon Comprehend Sentiment Analysis 

Amazon Comprehend can be used to perform sentiment analysis. You can accurately analyze customer interactions, including social media posts, reviews, customer interaction transcripts to improve your products and services.



This lab includes step-by-step instructions for performing sentiment analysis using Amazon Comprehend.

## Setup

Let's start by specifying:

* AWS region.
* The IAM role arn used to give access to Comprehend API and S3 bucket.
* The S3 bucket that you want to use for training and model data.


In [3]:

import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()

In [4]:
prefix = "sagemaker/sentiment-analysis"
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)
# customize to your bucket where you have stored the data

## Data
Let's start by uploading the dataset the sample data s3 bucket.The  sample dataset contains Amazon reviews taken from the larger dataset "Amazon reviews - Full", which was published with the article "Character-level Convolutional Networks for Text Classification" (Xiang Zhang et al., 2015). 

Now lets read this into a Pandas data frame and take a look.


In [6]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd 

data = pd.read_csv('./amazon-reviews.csv')   
pd.set_option('display.max_rows', 20)# Keep the output on one page
data

,Written in old English. It was very hard to read as I had to think through most sentences to figure out what was being said.
0,Thought I was getting a book received book on ...
1,this book was recommend 2 me from my neighbor....
2,I believe that this is a fantastic book for th...
3,I always liked this book. The kindle version i...
4,Hawthorne wrote a masterful work of historical...
...,...
30,a classic book where there is really nothing b...
31,"I had to read this book, which I had heard qui..."
32,This book was purchased for my daughter for a ...
33,A classic? Hardly. In it's time this book may ...


## Sentiment Analysis using Comprehend
In order to determine the overall emotional tone of text, we will use the DetectSentiment operation. To detect the sentiment in up to 25 documents in a batch, use the BatchDetectSentiment operation. For more information, see Using the Batch APIs. 


Lets check a plain text example to begin. 

Steps:
* Use boto3 to initialize the comprehend client
* Define the sample text 
* Called the detect_sentiment API and pass in the text as the input parameter. 

In [10]:
import boto3
import json

comprehend = boto3.client(service_name='comprehend', region_name=region)
                
text = "It is raining today in Seattle"

print('Calling DetectSentiment')
print(json.dumps(comprehend.detect_sentiment(Text=text, LanguageCode='en'), sort_keys=True, indent=4))
print('End of DetectSentiment\n')


Calling DetectSentiment
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "162",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 01 Jun 2021 17:52:15 GMT",
            "x-amzn-requestid": "5d26525d-82be-4b50-8e18-c675629a9205"
        },
        "HTTPStatusCode": 200,
        "RequestId": "5d26525d-82be-4b50-8e18-c675629a9205",
        "RetryAttempts": 0
    },
    "Sentiment": "NEUTRAL",
    "SentimentScore": {
        "Mixed": 9.628861880628392e-05,
        "Negative": 0.30989840626716614,
        "Neutral": 0.6552183032035828,
        "Positive": 0.03478698432445526
    }
}
End of DetectSentiment



Now lets use the API for our sample dataset and check the response 

In [16]:
for index, row in data.iterrows():
    print(row[0])
    print("\n")
    print(json.dumps(comprehend.detect_sentiment(Text=row[0], LanguageCode='en'), sort_keys=True, indent=4))

Thought I was getting a book received book on tape. a little deceiving. Rather than that the process went as expected. If it had been what we wanted would have been great.


{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "166",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 01 Jun 2021 17:57:48 GMT",
            "x-amzn-requestid": "fa6fedf3-cab6-4fba-ae5e-77f586eba6cc"
        },
        "HTTPStatusCode": 200,
        "RequestId": "fa6fedf3-cab6-4fba-ae5e-77f586eba6cc",
        "RetryAttempts": 0
    },
    "Sentiment": "NEGATIVE",
    "SentimentScore": {
        "Mixed": 0.029959602281451225,
        "Negative": 0.9654799103736877,
        "Neutral": 0.002631053328514099,
        "Positive": 0.0019294769736006856
    }
}
this book was recommend 2 me from my neighbor. but i dont think that dis is dat gud of a book 2 recommend... i didnt like it 2 much but...


{
    "ResponseMetadata": {
        "HTTPHeaders": {
